In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torchvision.models import densenet121, DenseNet121_Weights, densenet169, densenet161, DenseNet161_Weights, DenseNet169_Weights, densenet201, DenseNet201_Weights
from torchvision.transforms import ToTensor, Lambda
from PIL import Image
import pandas as pd

In [2]:
# Set device to GPU if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [6]:
# Define transforms for data preprocessing
normMean = [ 0.485, 0.456, 0.406 ]
normStd = [ 0.229, 0.224, 0.225 ]

valTransform =  {
    'Ten_Crop':
    transforms.Compose([
        transforms.Resize(256),
        transforms.TenCrop(224),
        transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
        transforms.Lambda(lambda crops: torch.stack([transforms.Normalize(mean=normMean, std=normStd)(crop) for crop in crops])),
]),
    'Single_Crop':
    transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=normMean, std=normStd),
])   

}
models = {
    'densenet121':[densenet121, DenseNet121_Weights.DEFAULT],
    'densenet161':[densenet161, DenseNet161_Weights.DEFAULT],
    'densenet169':[densenet169, DenseNet169_Weights.DEFAULT],
    'densenet201':[densenet201, DenseNet201_Weights.DEFAULT]
}

In [7]:
df = pd.DataFrame(columns=['Model', 'top-1', 'top-5', 'Crop'])
for k in models.keys():
    m = models[k][0]
    w = models[k][1]
    # Load pre-trained DenseNet model
    model = m(weights=w)
    model.to(device)
    # Ensure model is in evaluation mode
    model.eval()
    
    # Load ImageNet validation dataset
    for vt in valTransform.keys():
        valset = torchvision.datasets.ImageFolder(root='/kaggle/input/imagenet1kvalidation/val/', transform=valTransform[vt])
        if vt == 'Single_Crop':
            valloader = torch.utils.data.DataLoader(valset, batch_size=128, shuffle=False)
        elif vt == 'Ten_Crop':
            valloader = torch.utils.data.DataLoader(valset, batch_size=32, shuffle=False)
            
        print(f'Model: {k}, Crop: {vt} is evaluting...')
        
        # Evaluate model on validation set
        correct_1 = 0
        correct_5 = 0
        total = 0
        with torch.no_grad():
            for images, labels in valloader:
                images = images.to(device)
                labels = labels.to(device)
                
                if vt == 'Single_Crop':
                    outputs = model(images)
                    _, pred = outputs.topk(5, 1, True, True)
                    pred = pred.t()
                    correct = pred.eq(labels.view(1, -1).expand_as(pred))
                    
                elif vt == 'Ten_Crop':
                    outputs = model(images.view(-1, 3, 224, 224))  # Reshape images
                    outputs = outputs.view(images.size(0), -1, 1000)  # Reshape outputs
                    _, pred = outputs.topk(5, 1, True, True)
                    pred = pred.transpose(1, 0)
                    correct = pred.eq(labels.view(1, -1, 1).expand_as(pred))
                    
                # Compute top 5
                correct_5 += correct[:5].reshape(-1).float().sum(0)

                # Compute top1 
                correct_1 += correct[:1].reshape(-1).float().sum(0)
                
                if vt == 'Single_Crop':
                    total += labels.size(0)
                elif vt == 'Ten_Crop':
                    total += labels.size(0) * 10
              
        top1_error_rate = 1 - (correct_1 / total).cpu().numpy()
        top5_error_rate = 1 - (correct_5 / total).cpu().numpy()
        new_data = {'Model': [k], 'top-1': [top1_error_rate], 'top-5': [top5_error_rate], 'Crop': [vt]}
        df = pd.concat([df, pd.DataFrame(new_data)], ignore_index=True)

Model: densenet121, Crop: Ten_Crop is evaluting...
Model: densenet121, Crop: Single_Crop is evaluting...


Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth
100%|██████████| 110M/110M [00:00<00:00, 227MB/s]  


Model: densenet161, Crop: Ten_Crop is evaluting...
Model: densenet161, Crop: Single_Crop is evaluting...


Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /root/.cache/torch/hub/checkpoints/densenet169-b2777c0a.pth
100%|██████████| 54.7M/54.7M [00:00<00:00, 235MB/s] 


Model: densenet169, Crop: Ten_Crop is evaluting...
Model: densenet169, Crop: Single_Crop is evaluting...


Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/hub/checkpoints/densenet201-c1103571.pth
100%|██████████| 77.4M/77.4M [00:00<00:00, 229MB/s] 


Model: densenet201, Crop: Ten_Crop is evaluting...
Model: densenet201, Crop: Single_Crop is evaluting...


In [9]:
def format_float(value):
    if isinstance(value, float):
        return "{:.2%}".format(value)
    return value

df = df.applymap(format_float)

df

,Model,top-1,top-5,Crop
0,densenet121,23.01%,7.02%,Ten_Crop
1,densenet121,25.56%,8.03%,Single_Crop
2,densenet161,21.11%,5.68%,Ten_Crop
3,densenet161,22.89%,6.44%,Single_Crop
4,densenet169,22.02%,6.74%,Ten_Crop
5,densenet169,24.41%,7.20%,Single_Crop
6,densenet201,21.60%,5.63%,Ten_Crop
7,densenet201,23.11%,6.63%,Single_Crop
